In [1]:
!pip install selenium webdriver-manager

  Using cached selenium-4.39.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached urllib3-2.6.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached trio-0.32.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.3.2-py3-none-any.whl.metadata (5.2 kB)
Using cached selenium-4.39.0-py3-none-any.whl (9.7 MB)
Using cached trio-0.32.0-py3-none-any.whl (512 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
Using cached urllib3-2.6.2-py3-none-any.whl (131 kB)
Using cached certifi-2025.11.12-py3-none-any.whl (159 kB)
Using cached outcome-1.3.0.post0-py2.p

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kubernetes 34.1.0 requires urllib3<2.4.0,>=1.24.2, but you have urllib3 2.6.2 which is incompatible.


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# 1. 브라우저 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-logging"])

print("ㅇㅅㅇ: 잠자는 현대카드 사이트를 깨우러 갑니다...")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 2. 페이지 이동
url = "https://www.hyundaicard.com/cpu/ug/CPUUG1001_02.hc"
driver.get(url)
driver.implicitly_wait(10) # 페이지 요소들이 준비될 때까지 최대 10초 대기

# ID 설정
combo_box_id = "rglnClsfCd"
search_btn_id = "searchBtn"

try:
    # ---------------------------------------------------------------
    # 1단계: 약관 이름들만 먼저 싹 다 가져오기 (에러 방지용)
    # ---------------------------------------------------------------
    select_element = driver.find_element(By.ID, combo_box_id)
    selector = Select(select_element)
    
    # 옵션의 텍스트만 리스트로 미리 저장해둡니다.
    # 이렇게 안 하면 반복문 돌다가 "요소가 사라졌어요(Stale)" 에러가 납니다.
    option_texts = [opt.text for opt in selector.options]
    total_count = len(option_texts)
    
    print(f"ㅇㅅㅇ: 총 {total_count}개의 약관 목록을 확보했습니다!")
    
    full_text_data = "" 

    # ---------------------------------------------------------------
    # 2단계: 하나씩 "강제로" 선택하고 수집하기
    # ---------------------------------------------------------------
    for i in range(total_count):
        current_option_name = option_texts[i]
        
        # 빈 칸이거나 '선택' 같은 의미 없는 항목이면 건너뛰기
        if not current_option_name.strip():
            continue

        print(f"\n[{i+1}/{total_count}] '{current_option_name}' 처리 중...")

        try:
            # (1) 콤보박스 다시 찾기 (새로고침 될 수 있으므로 매번 찾음)
            select_element = driver.find_element(By.ID, combo_box_id)
            selector = Select(select_element)
            
            # (2) ★핵심★ 강제 재선택!
            # 이미 선택되어 있어도 다시 누릅니다. 그래야 사이트가 인식합니다.
            selector.select_by_index(i)
            time.sleep(1) # 선택 후 잠깐 숨 고르기

            # (3) ★핵심★ 조회 버튼 "자바스크립트"로 강제 클릭
            # 그냥 .click()이 안 먹힐 때 쓰는 필살기입니다.
            search_btn = driver.find_element(By.ID, search_btn_id)
            driver.execute_script("arguments[0].click();", search_btn)
            
            # (4) 데이터 로딩 대기 (3초)
            # "등록된 내역이 없습니다"가 사라지고 내용이 뜰 시간을 줍니다.
            time.sleep(3) 

            # (5) 내용 긁어오기
            # 내용이 없으면(등록된 내역...) 건너뛰어야 할 수도 있지만 일단 다 긁습니다.
            body_element = driver.find_element(By.ID, "container")
            body_text = body_element.text
            
            # 만약 "등록된 내역이 없습니다"만 있다면 수집 실패로 간주하거나 로그 남기기
            if "등록된 내역이 없습니다" in body_text and len(body_text) < 200:
                print(f"   ⚠️ 내용 없음 (아직 로딩 안 됨? 재시도 필요할 수 있음)")
            else:
                print(f"   ✅ 수집 성공! (길이: {len(body_text)}자)")

            # 저장용 변수에 추가
            full_text_data += f"\n{'='*70}\n"
            full_text_data += f"제목: {current_option_name}\n"
            full_text_data += f"{'='*70}\n\n"
            full_text_data += body_text + "\n\n"

        except Exception as e:
            print(f"   ❌ 에러 발생: {e}")
            # 에러가 나도 다음 약관으로 계속 진행
            continue

    # 3. 파일 저장
    save_file_name = "hyundai_card_real_terms.txt"
    with open(save_file_name, "w", encoding="utf-8") as f:
        f.write(full_text_data)
        
    print(f"\nㅇㅅㅇ: 수집 완료! '{save_file_name}' 파일을 확인해주세요.")

except Exception as e:
    print("치명적 오류:", e)

ㅇㅅㅇ: 잠자는 현대카드 사이트를 깨우러 갑니다...
ㅇㅅㅇ: 총 54개의 약관 목록을 확보했습니다!

ㅇㅅㅇ: 수집 완료! 'hyundai_card_real_terms.txt' 파일을 확인해주세요.


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time

# 1. 브라우저 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-logging"])

print("ㅇㅅㅇ: 현대카드 사이트 접속 중...")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 2. 페이지 이동
url = "https://www.hyundaicard.com/cpu/ug/CPUUG1001_02.hc"
driver.get(url)
driver.implicitly_wait(10)

# ID 설정
combo_box_id = "rglnClsfCd"
search_btn_id = "searchBtn"

try:
    # ---------------------------------------------------------------
    # 1단계: 약관 이름 가져오기 (수정됨: 숨겨진 텍스트도 읽기)
    # ---------------------------------------------------------------
    select_element = driver.find_element(By.ID, combo_box_id)
    selector = Select(select_element)
    
    # [수정 포인트] .text 대신 .get_attribute("innerText") 사용!
    # 이렇게 하면 화면에 안 보여도 HTML에 있는 글자를 무조건 가져옵니다.
    option_texts = [opt.get_attribute("innerText") for opt in selector.options]
    total_count = len(option_texts)
    
    print(f"ㅇㅅㅇ: 총 {total_count}개의 약관을 찾았습니다. 수집 시작!")
    
    full_text_data = "" 

    # ---------------------------------------------------------------
    # 2단계: 순서대로 수집
    # ---------------------------------------------------------------
    for i in range(total_count):
        current_option_name = option_texts[i]
        
        # 혹시라도 이름이 진짜 비어있으면 "이름없음"으로 처리하고 진행
        if not current_option_name.strip():
            current_option_name = f"약관_{i+1}"

        print(f"\n[{i+1}/{total_count}] '{current_option_name}' 선택 및 수집 중...")

        try:
            # (1) 콤보박스 다시 잡기
            select_element = driver.find_element(By.ID, combo_box_id)
            selector = Select(select_element)
            
            # (2) 선택 변경
            selector.select_by_index(i)
            time.sleep(0.5) # 살짝 대기

            # (3) 조회 버튼 '자바스크립트'로 강제 클릭
            search_btn = driver.find_element(By.ID, search_btn_id)
            driver.execute_script("arguments[0].click();", search_btn)
            
            # (4) 내용 로딩 대기 (3초)
            time.sleep(3) 

            # (5) 내용 긁어오기
            # container 전체 텍스트 가져오기
            body_element = driver.find_element(By.ID, "container")
            body_text = body_element.text
            
            # 데이터 검증 로그
            if len(body_text) < 100:
                print(f"   ⚠️ 내용이 너무 짧아요. (로딩 실패 가능성)")
            else:
                print(f"   ✅ 수집 성공! (글자수: {len(body_text)}자)")

            # (6) 저장 변수에 추가
            full_text_data += f"\n{'='*70}\n"
            full_text_data += f"제목: {current_option_name}\n"
            full_text_data += f"{'='*70}\n\n"
            full_text_data += body_text + "\n\n"

        except Exception as e:
            print(f"   ❌ 에러 발생 ({current_option_name}): {e}")
            continue

    # 3. 파일 저장
    save_file_name = "hyundai_card_real_terms.txt"
    with open(save_file_name, "w", encoding="utf-8") as f:
        f.write(full_text_data)
        
    print(f"\nㅇㅅㅇ: 진짜 완료! '{save_file_name}' 파일을 확인해주세요!")

except Exception as e:
    print("치명적 오류:", e)

ㅇㅅㅇ: 현대카드 사이트 접속 중...
ㅇㅅㅇ: 총 54개의 약관을 찾았습니다. 수집 시작!

[1/54] '현대카드 개인회원 약관' 선택 및 수집 중...
   ⚠️ 내용이 너무 짧아요. (로딩 실패 가능성)

[2/54] '온라인서비스 회원약관' 선택 및 수집 중...
   ❌ 에러 발생 (온라인서비스 회원약관): Message: element not interactable: Element is not currently visible and may not be manipulated
  (Session info: chrome=143.0.7499.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementnotinteractableexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0xc012d3
	0xc01314
	0x9ee6dd
	0x9f5323
	0x9f76c1
	0x9f7758
	0xa3cdfd
	0xa2fbc4
	0xa5b4ec
	0xa2ec34
	0xa5b6a4
	0xa7db13
	0xa5b2e6
	0xa2d321
	0xa2e1d4
	0xe55314
	0xe508cb
	0xe6d1aa
	0xc1b1d8
	0xc231dd
	0xc095d8
	0xc09799
	0xbf3b28
	0x759a5d49
	0x7766d5db
	0x7766d561


[3/54] '전자상거래(인터넷사이버몰) 약관' 선택 및 수집 중...
   ❌ 에러 발생 (전자상거래(인터넷사이버몰) 약관): Message: element not interactable: Element is not currently visible and may not be manipulated
  (Session info: chrome=143.0.7499

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time

# 1. 브라우저 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-logging"])

print("ㅇㅅㅇ: 현대카드 사이트(수술 모드) 접속 중...")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 2. 페이지 이동
url = "https://www.hyundaicard.com/cpu/ug/CPUUG1001_02.hc"
driver.get(url)
driver.implicitly_wait(10)

# ID 설정
combo_box_id = "rglnClsfCd"
search_btn_id = "searchBtn"

try:
    # ---------------------------------------------------------------
    # 1단계: 약관의 '번호(Value)'와 '이름(Text)' 미리 확보하기
    # ---------------------------------------------------------------
    # 클릭은 안 되더라도 정보는 읽을 수 있어요.
    select_element = driver.find_element(By.ID, combo_box_id)
    options = select_element.find_elements(By.TAG_NAME, "option")
    
    # (약관이름, 약관번호) 쌍으로 리스트에 저장합니다.
    # 예: [('현대카드 개인회원 약관', '11'), ('온라인서비스...', '9')]
    # get_attribute("value")가 진짜 핵심이에요!
    terms_list = []
    for opt in options:
        text = opt.get_attribute("innerText").strip()
        val = opt.get_attribute("value")
        if text and val: # 빈칸 아니면 저장
            terms_list.append((text, val))
            
    total_count = len(terms_list)
    print(f"ㅇㅅㅇ: 총 {total_count}개의 약관 코드를 확보했습니다! 강제 수집 시작!")
    
    full_text_data = "" 

    # ---------------------------------------------------------------
    # 2단계: 자바스크립트로 값 강제 변경 후 조회
    # ---------------------------------------------------------------
    for i in range(total_count):
        name, value = terms_list[i]
        
        print(f"\n[{i+1}/{total_count}] '{name}' (코드:{value}) 수집 시도...")

        try:
            # (1) ★핵심 필살기★ 자바스크립트로 콤보박스 값 강제 변경
            # 마우스로 선택하는 게 아니라, 코드로 값을 그냥 넣어버립니다.
            driver.execute_script(f"document.getElementById('{combo_box_id}').value = '{value}';")
            
            # (2) 값 변경 후 '조회' 버튼 강제 클릭
            search_btn = driver.find_element(By.ID, search_btn_id)
            driver.execute_script("arguments[0].click();", search_btn)
            
            # (3) 로딩 대기 (충분히 3~4초)
            time.sleep(3) 

            # (4) 내용 긁어오기
            body_element = driver.find_element(By.ID, "container")
            body_text = body_element.text
            
            # 내용 검증
            if len(body_text) < 200: # 내용이 너무 짧으면
                 print(f"   ⚠️ 경고: 내용이 비어있거나 로딩되지 않았습니다. (재시도 로직 필요할 수 있음)")
                 # 혹시 모르니 2초 더 기다리고 다시 긁어보기
                 time.sleep(2)
                 body_text = driver.find_element(By.ID, "container").text

            print(f"   ✅ 수집 성공! (길이: {len(body_text)}자)")

            # (5) 저장
            full_text_data += f"\n{'='*70}\n"
            full_text_data += f"제목: {name}\n"
            full_text_data += f"{'='*70}\n\n"
            full_text_data += body_text + "\n\n"

        except Exception as e:
            print(f"   ❌ 에러 발생: {e}")
            continue

    # 3. 파일 저장
    save_file_name = "hyundai_card_final_terms.txt"
    with open(save_file_name, "w", encoding="utf-8") as f:
        f.write(full_text_data)
        
    print(f"\nㅇㅅㅇ: 수술 성공! '{save_file_name}' 파일을 확인해주세요!")

except Exception as e:
    print("치명적 오류:", e)

ㅇㅅㅇ: 현대카드 사이트(수술 모드) 접속 중...
ㅇㅅㅇ: 총 54개의 약관 코드를 확보했습니다! 강제 수집 시작!

[1/54] '현대카드 개인회원 약관' (코드:11) 수집 시도...
   ⚠️ 경고: 내용이 비어있거나 로딩되지 않았습니다. (재시도 로직 필요할 수 있음)
   ✅ 수집 성공! (길이: 29자)

[2/54] '온라인서비스 회원약관' (코드:9) 수집 시도...
   ⚠️ 경고: 내용이 비어있거나 로딩되지 않았습니다. (재시도 로직 필요할 수 있음)
   ✅ 수집 성공! (길이: 29자)

[3/54] '전자상거래(인터넷사이버몰) 약관' (코드:10) 수집 시도...
   ⚠️ 경고: 내용이 비어있거나 로딩되지 않았습니다. (재시도 로직 필요할 수 있음)
   ✅ 수집 성공! (길이: 29자)

[4/54] '체크카드 개인회원 약관' (코드:12) 수집 시도...
   ⚠️ 경고: 내용이 비어있거나 로딩되지 않았습니다. (재시도 로직 필요할 수 있음)
   ✅ 수집 성공! (길이: 29자)

[5/54] '여신금융회사 표준 여신거래기본약관' (코드:16) 수집 시도...
   ⚠️ 경고: 내용이 비어있거나 로딩되지 않았습니다. (재시도 로직 필요할 수 있음)
   ✅ 수집 성공! (길이: 29자)

[6/54] '신용카드 가맹점 표준약관' (코드:17) 수집 시도...
   ⚠️ 경고: 내용이 비어있거나 로딩되지 않았습니다. (재시도 로직 필요할 수 있음)
   ✅ 수집 성공! (길이: 29자)

[7/54] '포인트 결제전환 이용약관' (코드:18) 수집 시도...
   ⚠️ 경고: 내용이 비어있거나 로딩되지 않았습니다. (재시도 로직 필요할 수 있음)
   ✅ 수집 성공! (길이: 29자)

[8/54] '현대카드 세이브(포인트 연계 할부서비스) 이용 약관' (코드:19) 수집 시도...
   ⚠️ 경고: 내용이 비어있거나 로딩되지 않았습니다. (재시도 로직 필요할 수 있음)
   ✅ 수집 성공! (길이: 29자)

[9/54] '통신

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time


options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-logging"])

print("ㅇㅅㅇ: 현대카드 사이트 접속 중...")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

url = "https://www.hyundaicard.com/cpu/ug/CPUUG1001_02.hc"
driver.get(url)

combo_box_id = "rglnClsfCd"
search_btn_id = "searchBtn"

# 1) 옵션 목록은 <select>에서 "읽기만" 한다 (선택은 커스텀 UI로)
select_el = wait.until(EC.presence_of_element_located((By.ID, combo_box_id)))
opt_els = select_el.find_elements(By.TAG_NAME, "option")

terms = []
for opt in opt_els:
    val = (opt.get_attribute("value") or "").strip()
    name = (opt.get_attribute("innerText") or "").strip()
    if not val:
        continue
    if not name:
        name = f"약관_{val}"
    terms.append((val, name))

print(f"ㅇㅅㅇ: 총 {len(terms)}개의 약관을 찾았습니다. 수집 시작!")

full_text_data = ""

def _norm(s: str) -> str:
    return " ".join((s or "").split())

def pick_term_by_custom_ui(value: str, name: str):
    btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".select_wrap.rglnClsfCd .btn_select")))
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    driver.execute_script("arguments[0].click();", btn)

    ul = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".select_wrap.rglnClsfCd .ul_select")))

    # ✅ 항상 맨 위로 초기화 (스크롤 위치 기억 방지)
    driver.execute_script("arguments[0].scrollTop = 0;", ul)
    time.sleep(0.05)

    found = False
    last_scroll_top = -1

    for _ in range(80):  # 60→80 약간 여유
        candidates = ul.find_elements(By.CSS_SELECTOR, f"a[href='#{value}']")
        if candidates:
            el = candidates[0]
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
            time.sleep(0.05)
            driver.execute_script("arguments[0].click();", el)
            found = True
            break

        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].clientHeight;", ul)
        time.sleep(0.05)

        scroll_top = driver.execute_script("return arguments[0].scrollTop;", ul)
        if scroll_top == last_scroll_top:
            break
        last_scroll_top = scroll_top

    if not found:
        raise Exception(f"드롭다운에서 옵션을 찾지 못했어요: value={value}, name={name}")

    # ✅ 완전일치 대신 "포함"으로 완화
    wait.until(lambda d: _norm(name) in _norm(d.find_element(By.CSS_SELECTOR, ".select_wrap.rglnClsfCd .btn_select").text))

def click_search_and_wait_dom_change():
    before = driver.find_element(By.ID, "container").text
    btn = wait.until(EC.element_to_be_clickable((By.ID, search_btn_id)))
    driver.execute_script("arguments[0].click();", btn)
    wait.until(lambda d: d.find_element(By.ID, "container").text != before)

for i, (val, name) in enumerate(terms, start=1):
    print(f"\n[{i}/{len(terms)}] '{name}' 선택 및 수집 중...")

    try:
        pick_term_by_custom_ui(val, name)
        click_search_and_wait_dom_change()

        # 일단 주인님 코드처럼 container 전체 텍스트를 읽음
        body_text = driver.find_element(By.ID, "container").text.strip()

        # 디버그용: 선택별 HTML 저장 (본문이 iframe/pdf/링크면 여기서 바로 확인 가능)
        with open(f"debug_{i:02d}_{val}.html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)

        if len(body_text) < 200:
            print("   ⚠️ 내용이 짧아요. (본문이 다른 영역/iframe/pdf/링크일 가능성)")
        else:
            print(f"   ✅ 수집 성공! (글자수: {len(body_text)}자)")

        full_text_data += "\n" + ("=" * 70) + "\n"
        full_text_data += f"제목: {name}\n"
        full_text_data += ("=" * 70) + "\n\n"
        full_text_data += body_text + "\n\n"

    except Exception as e:
        print(f"   ❌ 에러 발생 ({name}): {e}")
        continue

save_file_name = "hyundai_card_real_terms.txt"
with open(save_file_name, "w", encoding="utf-8") as f:
    f.write(full_text_data)

print(f"\nㅇㅅㅇ: 완료! '{save_file_name}' 파일을 확인해주세요! 😺")


ㅇㅅㅇ: 현대카드 사이트 접속 중...
ㅇㅅㅇ: 총 54개의 약관을 찾았습니다. 수집 시작!

[1/54] '현대카드 개인회원 약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 40239자)

[2/54] '온라인서비스 회원약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 8054자)

[3/54] '전자상거래(인터넷사이버몰) 약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 8805자)

[4/54] '체크카드 개인회원 약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 1673자)

[5/54] '여신금융회사 표준 여신거래기본약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 19744자)

[6/54] '신용카드 가맹점 표준약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 20780자)

[7/54] '포인트 결제전환 이용약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 5371자)

[8/54] '현대카드 세이브(포인트 연계 할부서비스) 이용 약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 6435자)

[9/54] '통신 세이브 서비스 이용약관' 선택 및 수집 중...
   ⚠️ 내용이 짧아요. (본문이 다른 영역/iframe/pdf/링크일 가능성)

[10/54] '휴대폰 인증 약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 51251자)

[11/54] '선불카드 표준약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 10735자)

[12/54] '가상카드번호서비스 약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 5862자)

[13/54] 'PIN번호 결제서비스 이용약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 6957자)

[14/54] '일반결제 서비스 이용약관' 선택 및 수집 중...
   ✅ 수집 성공! (글자수: 5319자)

[15/54] '간편결제 서비스 이용약관' 선택 및 수집 